In [1]:
!pip install langchain google-cloud-aiplatform rich

     ---------------------------------------- 1.2/1.2 MB 15.1 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 28.0 MB/s eta 0:00:00
     ------------------------------------- 239.4/239.4 kB 15.3 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB ? eta 0:00:00
     ---------------------------------------- 114.6/114.6 kB ? eta 0:00:00
     -------------------------------------- 120.5/120.5 kB 6.9 MB/s eta 0:00:00
     ------------------------------------- 321.3/321.3 kB 19.4 MB/s eta 0:00:00
     ------------------------------------- 219.5/219.5 kB 13.9 MB/s eta 0:00:00
  Using cached protobuf-4.23.3-cp310-abi3-win_amd64.whl (422 kB)
     ---------------------------------------- 1.3/1.3 MB 16.5 MB/s eta 0:00:00
     ---------------------------------------- 48.1/48.1 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 36.7 MB/s eta 0:00:00
     ---------------------------------------- 87.5/87.5 kB 4.8 MB/s eta

# Imports

In [2]:
import os
import requests
from google.cloud import aiplatform
from langchain.agents import (AgentExecutor, AgentType, BaseSingleActionAgent,
                              Tool, initialize_agent)
from langchain.llms import VertexAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from pydantic import BaseModel, Field
from rich import print
from rich.console import Console
from rich.progress import Progress, track
from rich.table import Table

# Google Authentication

In [4]:
tool_url = None #!TODO: set this to the tool url if using Colab
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    import dotenv
    dotenv.load_dotenv()
    tool_url = str(os.getenv('TOOL_URL'))
    pass

In [5]:
#! Change the following to your own project and location
aiplatform.init(project="aerobic-gantry-387923", location="us-central1")
llm = VertexAI()

# Vector Stores

In [ ]:
#TODO: Implement a vector stores

# Tools Function

In [6]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

def request_error_handler(url: str, params={}) -> str:
    try:
        response = requests.get(url, params)
        response.raise_for_status()  # Raise an HTTPError if the status code is not in the 200 range
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        return e

## Courses

In [7]:
def get_courses_from_query(query: str) -> str:
    url = f"https://{tool_url}/get-possible-courses/{query}"
    return request_error_handler(url)


## Degree

In [8]:
def get_degrees_from_query(query: str) -> str:
    url = f"https://{tool_url}/get-possible-degrees/{query}"
    return request_error_handler(url)
print(
    get_degrees_from_query("CS")
)

{
    'items': [
        {
            'title': 'Computer Science (BS) - UT Dallas 2022 Undergraduate Catalog ...',
            'link': 'https://catalog.utdallas.edu/2022/undergraduate/programs/ecs/computer-science',
            'snippet': 'Mar 25, 2022 ... The Computer Science program continues with courses in advanced data 
structures, programming languages, telecommunications networks, and\xa0...'
        },
        {
            'title': 'Computer Science Program - UT Dallas 2022 Graduate Catalog ...',
            'link': 'https://catalog.utdallas.edu/2022/graduate/programs/ecs/computer-science',
            'snippet': "Apr 13, 2022 ... CS 5349 Automata Theory. CS 5390 Computer Networks. The University's 
general degree requirements are discussed on the Graduate Policies and\xa0..."
        },
        {
            'title': 'Computer Science (BS) - Four-Year Degree Plan (Example) - UT ...',
            'link': 'https://catalog.utdallas.edu/2022/undergraduate/programs/ecs/computer-science/four-year',
            'snippet': 'May 21, 2022 ... CS 2336, 9, 3. Total, 19, Total, 16. Junior Year. Semester 5 - Fall, 
Notes, Preferred, Core, SCH, Semester 6 - Spring, Notes, Preferred\xa0...'
        },
        {
            'title': 'Minors - Erik Jonsson School of Engineering and Computer Science ...',
            'link': 'https://catalog.utdallas.edu/2022/undergraduate/programs/ecs/minors',
            'snippet': 'Feb 11, 2022 ... Minor in Computer Science. 21 semester credit hours. Students majoring in 
Computer Engineering, Computer Science or Software Engineering cannot\xa0...'
        },
        {
            'title': 'Computer Science - BS - Erik Jonsson School of Engineering and ...',
            'link': 'https://catalog.utdallas.edu/2020/undergraduate/programs/ecs/computer-science',
            'snippet': 'Jun 18, 2020 ... The mission of the Department of Computer Science is to prepare 
undergraduate and graduate students for productive careers in industry, academia\xa0...'
        },
        {
            'title': 'Erik Jonsson School of Engineering and Computer Science',
            'link': 'https://catalog.utdallas.edu/2020-undergraduate/programs/ecs/computer-science/four-year',
            'snippet': 'May 26, 2020 ... CS 1200 Introduction to Computer Science and Software Engineering, 2, CS 
2305 Discrete Mathematics for Computing I, 3.'
        },
        {
            'title': 'Department of Computer Science - UT Dallas 2020 Graduate Catalog',
            'link': 'https://catalog.utdallas.edu/2020/graduate/programs/ecs/computer-science',
            'snippet': 'Oct 14, 2020 ... CS 3341 (SE 3341) Probability and Statistics in Computer Science and 
Software Engineering. CS 5390 Computer Networks.'
        },
        {
            'title': 'Certificates - Erik Jonsson School of Engineering and Computer ...',
            'link': 'https://catalog.utdallas.edu/2022/undergraduate/programs/ecs/certificates',
            'snippet': 'Aug 2, 2021 ... CS 4393 Computer and Network Security. CS 4398 Digital Forensics. Students 
that complete the Minor in Information Assurance will not be\xa0...'
        },
        {
            'title': 'Software Engineering (BS) - UT Dallas 2022 Undergraduate Catalog',
            'link': 'https://catalog.utdallas.edu/2022/undergraduate/programs/ecs/software-engineering',
            'snippet': 'Jul 22, 2022 ... CS 1200 Introduction to Computer Science and Software Engineering ... SE 
guided electives are 4000 level CS/SE courses approved by the\xa0...'
        },
        {
            'title': 'Data Science - BS - Four Year - UT Dallas 2020 Undergraduate ...',
            'link': 'https://catalog.utdallas.edu/2020-undergraduate/programs/data-science/four-year',
            'snippet': 'May 27, 2020 ... This is an example only. Please see advisor to develop individual 
four-year plan. Freshman Year. Fall Semester, SCH, Spring Semester, SCH. CS\xa0...'
        }
    ]
}

## Utdallas Search

In [9]:
def utdallas_search(query: str) -> str:
    url = f"https://{tool_url}/search/{query}"
    return request_error_handler(url)

def access_utdallas_site(url_to_search: str) -> str:
    url = f"https://{tool_url}/access/"
    params = {
        "url": url_to_search
    }
    return request_error_handler(url, params)



# Tools NAME and description

In [10]:
tools = [
    Tool(
        name = "UTD Degree API",
        func = get_degrees_from_query,
        description = "Suppose you have a keyword that sound like a UT Dallas' Degrees, Majors, Minors, and Certificates. This tool will return possible url, title and snippet from the query from search engine.",
    ),
    Tool(
        name = "UTD Course API",
        func = get_courses_from_query,
        description = "Suppose you have a keyword that sound like a UT Dallas' courses. This tool will return course numbers/codes where that keyword exists in the course title.",
    ),

    Tool(
        name = "UTD Search",
        func = utdallas_search,
        description = "This tool should not be used and should be used only as a last resort; it will search every UT Dallas page. This function require a query string and return possible links and snippet contents. If you want to find classes or degrees, there are better tools."
    ),
    Tool(
        name = "Extract text from UTD Search url links",
        func = access_utdallas_site,
        description = "If you have a url string contain utdallas.edu, use this function to extract text content"
    ),

    ]

# Initialize Agents

## Zero Shot ReAcT Agents


In [11]:
'''
agent = initialize_agent(
    tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory = memory ,
    max_execution_time = 10 , return_intermediate_steps=True,
)
'''
memory = ConversationBufferMemory(memory_key="chat_history")
zero_shot_react_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    max_execution_time = 20 ,
)

## Plan and execute

In [12]:
from langchain.experimental import plan_and_execute
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools = tools , verbose=True)
plan_and_execute = PlanAndExecute (
    planner=planner,
    executor=executor,
    verbose=True,
    max_execution_time = 20 ,
    return_intermediate_steps=True,
)




# Testing Agents

## Tell me the Difference between a courses

In [13]:
response = zero_shot_react_agent(
        {
            "input": "What is the difference between Math 2417 and Math 2419"
        }
)
print(
    response["output"]
)




> Entering new  chain...
I don't know what Math 2417 and Math 2419 are
Action: UTD Course API
Action Input: Math 2417
Observation: {'course_ids': {'items': [{'title': 'Calculus I - UT Dallas 2022 Undergraduate Catalog - The University ...', 'link': 'https://catalog.utdallas.edu/2022/undergraduate/courses/math2417', 'snippet': 'MATH2417 - Calculus I. MATH 2417 Calculus I (4 semester credit hours) Functions, limits, continuity, differentiation; integration of function of one\xa0...'}, {'title': 'Calculus I - UT Dallas 2022 Graduate Catalog - The University of ...', 'link': 'https://catalog.utdallas.edu/2022/graduate/courses/math2417', 'snippet': 'MATH2417 - Calculus I. MATH 2417 Calculus I (4 semester credit hours) Functions, limits, continuity, differentiation; integration of function of one\xa0...'}, {'title': 'Calculus II - UT Dallas 2017 Undergraduate Catalog - The ...', 'link': 'https://catalog.utdallas.edu/2017/undergraduate/courses/math2419', 'snippet': 'MATH 2419 Calculus II (4

Math 2417 is Calculus I and Math 2419 is Calculus II

In [14]:
try:
  plan_execute_response = plan_and_execute(
    "What is the difference between Math 2417 and Math 2419"
  )
  print(
    plan_execute_response["output"]
  )
except Exception as e:
  print(e)







> Entering new  chain...
steps=[Step(value='Go to the course catalog'), Step(value='Search for Math 2417'), Step(value='Read the course description'), Step(value='Search for Math 2419'), Step(value='Read the course description'), Step(value='Compare the two course descriptions'), Step(value='Given the above steps taken, please respond to the users original question\n')]

> Entering new  chain...
Action:
```
{
  "action": "UTD Course API",
  "action_input": "course catalog"
}
```

Observation: {'course_ids': {'items': [{'title': 'Psychology - UT Dallas 2022 Undergraduate Catalog - The ...', 'link': 'https://catalog.utdallas.edu/2022/undergraduate/courses/psy', 'snippet': 'UT Dallas 2022 Undergraduate Catalog. ... PSY 3339 Educational Psychology (3 semester credit hours) This course focuses on the psychological foundations of\xa0...'}, {'title': 'Mathematical Science - UT Dallas 2023 Undergraduate Catalog ...', 'link': 'https://catalog.utdallas.edu/current/undergraduate/courses/math', 

Could not parse LLM output: Action:
```
{
  "action": UTD Course API,
  "action_input": "Math 2419"
}
```

## Contacting People at my University

In [ ]:
zero_shot_react_agent_response = zero_shot_react_agent(
    {
        "input": "What is the email to contact for Accounting advising?"
    }
)
print(
    zero_shot_react_agent_response["output"]
)




> Entering new  chain...
I don't know what the email to contact for Accounting advising is.
Action: UTD Search
Action Input: accounting advising
Observation: {'items': [{'title': 'Academic Advising | Naveen Jindal School of Management - Dallas', 'link': 'https://jindal.utdallas.edu/advising/', 'snippet': 'JSOM Academic Advising is here to help guide you by providing you with new student guidelines, registration steps, and graduation information. Read more.'}, {'title': 'MS Accounting Registration/CPA Advising Session - Comet Calendar', 'link': 'https://calendar.utdallas.edu/event/ms_accounting_registrationcpa_advising_session_3439', 'snippet': 'MS Accounting Registration/CPA Advising Session. Wednesday, September 2, 2020 at 12:00pm to 2:00pm. Virtual Event. To help you review your CPA eligibility\xa0...'}, {'title': 'Staff List | Naveen Jindal School of Management - Dallas', 'link': 'https://jindal.utdallas.edu/staff-list/', 'snippet': 'Oct 20, 2021 ... Van Dam. Program Manager, Unde

OutputParserException: ignored

## Standard definition in advising

In [ ]:
#TODO: Ask it what is a course, major, schedule planner